In [38]:
#!/usr/bin/python

import os
import pandas as pd

import yaml

# traverse root directory, and list directories as dirs and files as files
model_results = {'model':[], 'dataset':[], 'mode':[], 'path':[], 'value':[]}
for root, dirs, files in os.walk("../models/"):
    path = root.split(os.sep)
    for file in files:
        if file == "best_pred.txt":
            path = os.path.join(root, file)
            with open(os.path.join(root, 'parameters.txt'), 'r') as fp:
                parameters = yaml.safe_load(fp)
                
                model_results['model'].append(parameters['MODEL']['BACKBONE'])
                try:
                    mode = parameters['DATASET']['MODE']
                    if mode == "RGB_HHA":
                        mode += parameters['MODEL']['NAME'][:3]
                    model_results['mode'].append(mode)
                except Exception as e:
                    if "USE_DEPTH" in parameters['DATASET']:
                        if parameters['DATASET']['USE_DEPTH']:
                            model_results['mode'].append('RGBD')
                        else:
                            model_results['mode'].append('RGB')
                    else:        
                        if "rgbd" in parameters['TRAIN']['CHECKNAME']:
                            model_results['mode'].append('RGBD')
                        else:
                            model_results['mode'].append('RGB')

                model_results['dataset'].append(parameters['DATASET']['NAME'])
           
            with open(path, 'r') as fp:
                value = float(fp.readline())
                model_results['path'].append(path)
                model_results['value'].append(value)

In [79]:
pd.options.display.max_colwidth = 35
pd.options.display.max_rows = 100

df = pd.DataFrame(model_results)
df = df.sort_values('value', ascending=False)

with open('all_models.csv', 'w') as fp:
    df.to_csv(fp)

In [80]:
models = ['xception', 'resnet']
datasets = ['cityscapes', 'coco', 'sunrgbd']
modes = ['RGB', "RGBD", 'RGB_HHAmid', 'RGB_HHAlat']

with open('top_models.csv', 'w') as fp:
    header = True
    for model in models:
        for data in datasets:
            for mode in modes:
                if len(df[((df['model'] == model) & (df['dataset'] == data) & (df['mode'] == mode))].index) > 1:
                    row = df[((df['model'] == model) & (df['dataset'] == data) & (df['mode'] == mode))]
                    indices.append(row.index[0])
                    print(row.head(1))
                    row.head(1).to_csv(fp, header=header)
                    header = False

       model     dataset mode                                path     value
57  xception  cityscapes  RGB  ../models/temp/models/cityscape...  0.763971
       model     dataset  mode                                path     value
67  xception  cityscapes  RGBD  ../models/temp/models/cityscape...  0.780869
       model     dataset        mode                                path  \
41  xception  cityscapes  RGB_HHAmid  ../models/temp/models/cityscape...   

      value  
41  0.67721  
     model     dataset mode                                path    value
49  resnet  cityscapes  RGB  ../models/temp/models/cityscape...  0.75386
     model     dataset  mode                                path     value
64  resnet  cityscapes  RGBD  ../models/temp/models/cityscape...  0.769716
     model     dataset        mode                                path  \
38  resnet  cityscapes  RGB_HHAmid  ../models/temp/models/cityscape...   

       value  
38  0.772335  
     model     dataset        mode    

In [74]:
indices

[57, 67, 49, 64, 89, 31]